# 06: Procesamiento de Diputados y Biografías

**Propósito:** Este *notebook* toma todos los archivos `diputados_bio.csv` (crudos, por período) de `data/01_raw/`, y los transforma en un único archivo maestro de diputados limpio.

**Proceso:**
1.  **Carga y Consolidación:** Lee todos los archivos `diputados_bio.csv` y los une.
2.  **Limpieza y Estandarización:** Normaliza los campos extraídos por el LLM (ej. `universidad`, `maximo_nivel_educativo`) usando mapeos y *fuzzy matching*.
3.  **Deduplicación:** Crea un registro único por `Diputado.Id`, seleccionando la biografía de mayor calidad (mejor `match_score`).
4.  **Guardado:** Guarda el archivo maestro en `data/02_processed/`.

**Dependencias:**
* `data/01_raw/[periodo]/diputados_bio.csv` (Múltiples archivos)

**Salidas (Artifacts):**
* `data/02_processed/diputados_master_clean.parquet` (Un único archivo)

In [1]:
import pandas as pd
from pathlib import Path
import sys
import logging
from tqdm.notebook import tqdm # Para progress_apply
import numpy as np

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    from src.processing_utils import (
        load_all_bio_files, 
        standardize_education,
        standardize_civil_status,
        standardize_location,
        extract_last_colegio,
        standardize_career,
        create_age_features
    )
    from src.common_utils import normalize_string
except ImportError as e:
    logging.error(f"ERROR: No se pudieron importar las funciones desde /src. {e}")
    raise

# Registrar 'tqdm' con pandas
tqdm.pandas()

In [2]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"
DATA_DIR_PROCESSED = ROOT / "data" / "02_processed"

# Asegurarse que el directorio de salida exista
DATA_DIR_PROCESSED.mkdir(parents=True, exist_ok=True)

# Archivo de salida
OUTPUT_FILE = DATA_DIR_PROCESSED / "diputados_master_clean.parquet"

logging.info(f"Directorio Raw: {DATA_DIR_RAW}")
logging.info(f"Directorio Processed: {DATA_DIR_PROCESSED}")
logging.info(f"Archivo de Salida: {OUTPUT_FILE}")

2025-10-29 10:51:29,601 - INFO - Directorio Raw: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw
2025-10-29 10:51:29,604 - INFO - Directorio Processed: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed
2025-10-29 10:51:29,605 - INFO - Archivo de Salida: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed\diputados_master_clean.parquet


## 1. Carga y Consolidación

Cargamos todos los archivos `diputados_bio.csv` de todos los períodos en un solo DataFrame.

In [3]:
# Llamar a la función del módulo /src
df_full = load_all_bio_files(DATA_DIR_RAW)

if not df_full.empty:
    display(df_full.head())
    print(f"Dimensiones del DataFrame consolidado: {df_full.shape}")
else:
    logging.error("No se cargaron datos. Deteniendo el notebook.")
    # raise Exception("No se cargaron datos.")

2025-10-29 10:51:29,612 - INFO - Buscando archivos 'diputados_bio.csv'...
2025-10-29 10:51:29,614 - INFO - Encontrados 7 archivos. Cargando...
2025-10-29 10:51:29,689 - INFO - DataFrame consolidado creado con 929 filas.


,FechaInicio,FechaTermino,Diputado.Id,Diputado.Nombre,Diputado.Nombre2,Diputado.ApellidoPaterno,Diputado.ApellidoMaterno,Diputado.FechaNacimiento,Diputado.FechaDefucion,Diputado.RUT,...,madre,estado_civil,numero_total_hijos,colegios,universidad,carrera,maximo_nivel_educativo,trabajo,fuente_periodo,Distrito
0,2002-03-10,NaN,1,Mario,NaN,Acuña,Cisternas,NaN,NaN,NaN,...,María Cisterna Fuentealba,Casado/a,3.0,[],Universidad Austral de Chile,Ingeniero Agrónomo,Educación Universitaria,[],1998-2002,NaN
1,2002-03-10,NaN,3,Gustavo,NaN,Alessandri,Valdés,NaN,NaN,NaN,...,Verónica Balmaceda,Casado/a,3.0,['Colegio de los Sagrados Corazones de Manqueh...,Universidad de California,Maquinaria agrícola,Educación Universitaria,"['Obrero', 'Empresario independiente', 'Propie...",1998-2002,NaN
2,2002-03-10,NaN,8,Rafael,NaN,Arratia,Valdebenito,NaN,NaN,NaN,...,Olivia Valdebenito Cuevas,Casado/a,4.0,['Colegio Manuel León Prado'],Universidad de Chile,Médico cirujano,Educación Universitaria,"['Médico oftalmólogo', 'Director Médico de la ...",1998-2002,NaN
3,2002-03-10,NaN,10,Nelson,NaN,Avila,Contreras,NaN,NaN,NaN,...,María Olivia Contreras Chinchón,Casado,2.0,"['Colegio Las Carmelitas', 'Liceo de Hombres d...",Universidad de Chile,Administrador Público,Educación Universitaria,['Jefe de personal en Entel Chile'],1998-2002,NaN
4,2002-03-10,NaN,11,Francisco,NaN,Bartolucci,Johnston,NaN,NaN,NaN,...,Josefina Jhonston Miranda,Casado/a,5.0,['Colegio San Pedro Nolasco'],Universidad Católica de Valparaíso,Ciencias Jurídicas y Sociales,Educación Universitaria,['Profesor auxiliar de Derecho Romano'],1998-2002,NaN


Dimensiones del DataFrame consolidado: (929, 43)


## 2. Limpieza y Estandarización

Aplicamos las funciones de limpieza de `src/processing_utils.py` para normalizar los campos extraídos por el LLM.

In [12]:
if not df_full.empty:
    # --- Limpieza de Tipos ---
    df_processed = df_full.copy()
    logging.info("Limpiando tipos de datos...")
    df_processed['fecha_nacimiento'] = pd.to_datetime(df_full['fecha_nacimiento'], errors='coerce')
    df_processed['numero_total_hijos'] = pd.to_numeric(df_full['numero_total_hijos'], errors='coerce').astype('Int64')
    
    df_processed = standardize_education(df_processed)
    df_processed = standardize_career(df_processed)
    df_processed['fecha_nacimiento_llm'] = pd.to_datetime(df_processed['fecha_nacimiento'], errors='coerce')
    df_processed['fecha_nacimiento_api'] = pd.to_datetime(df_processed['Diputado.FechaNacimiento'], errors='coerce')
    df_processed['fecha_nac_clean'] = df_processed['fecha_nacimiento_api'].fillna(
            df_processed['fecha_nacimiento_llm']
        )
    df_processed['colegio_egreso_raw'] = df_processed['colegios'].apply(extract_last_colegio)
    df_processed['estado_civil_clean'] = df_processed['estado_civil'].progress_apply(lambda x: standardize_civil_status(pd.Series(x)))
    df_location_features = standardize_location(df_processed['lugar_nacimiento'])
    df_processed = df_processed.join(df_location_features)
    df_processed['colegio_egreso_merge_key'] = df_processed['colegio_egreso_raw'].apply(
        lambda x: np.nan if pd.isna(x) else normalize_string(x)
    )
    df_processed = create_age_features(df_processed, 'FechaInicio.1', 'fecha_nac_clean')
    
    logging.info("Procesamiento de campos finalizado.")
    display(df_processed[['universidad', 'universidad_clean', 'maximo_nivel_educativo', 'educacion_nivel_clean']].sample(10))
else:
    logging.warning("DataFrame vacío, saltando limpieza.")

2025-10-29 11:16:47,538 - INFO - Limpiando tipos de datos...
2025-10-29 11:16:47,543 - INFO - Estandarizando 'maximo_nivel_educativo'...
2025-10-29 11:16:47,551 - INFO - Estandarizando 'universidad' con estrategia 'map-once'...
2025-10-29 11:16:47,552 - INFO - Se encontraron 118 valores únicos de universidad.
2025-10-29 11:16:47,553 - INFO - Construyendo mapa de traducción (Manual + Fuzzy)...
Creando Mapa Fuzzy: 100%|██████████████████████████████████████████████████████████| 118/118 [00:00<00:00, 2301.22it/s]
2025-10-29 11:16:47,606 - INFO - Aplicando mapa a todas las filas...
2025-10-29 11:16:47,609 - INFO - Clasificando tipo de universidad...
2025-10-29 11:16:47,611 - INFO - Estandarizando 'carrera' con estrategia 'map-once'...
2025-10-29 11:16:47,612 - INFO - Se encontraron 212 valores únicos de carrera.
2025-10-29 11:16:47,613 - INFO - Construyendo mapa de traducción de carreras (Regex)...
Mapeando Carreras: 100%|██████████████████████████████████████████████████████████| 212/212 

  0%|          | 0/929 [00:00<?, ?it/s]

2025-10-29 11:16:48,796 - INFO - Creando features 'edad' y 'rango_etario'...
2025-10-29 11:16:48,804 - INFO - Procesamiento de campos finalizado.


,universidad,universidad_clean,maximo_nivel_educativo,educacion_nivel_clean
483,Universidad Diego Portales,Universidad Diego Portales,Educación Universitaria,Universitaria
742,Instituto Inacap,Universidad Tecnológica de Chile INACAP,Educación Universitaria,Universitaria
845,Universidad de Concepción,Universidad de Concepción,Educación Universitaria,Universitaria
4,Universidad Católica de Valparaíso,Universidad Católica de Valparaíso,Educación Universitaria,Universitaria
582,Universidad de Chile,Universidad de Chile,Educación Universitaria,Universitaria
466,Pontificia Universidad Católica de Chile,Pontificia Universidad Católica de Chile,Magíster,Magíster
359,Universidad de Chile,Universidad de Chile,Educación Universitaria,Universitaria
164,Universidad de Chile,Universidad de Chile,Educación Universitaria,Universitaria
257,Universidad Técnica Federico Santa María,Universidad Técnica Federico Santa María,Educación Universitaria,Universitaria
456,Escuela Nacional de Antropología e Historia de...,Universidad Extranjera,Educación Universitaria,Universitaria


In [14]:
if not df_processed.empty:
    
    # --- 1. (NUEVO) Consolidar Columnas Duplicadas ---
    logging.info("Consolidando columnas duplicadas (ej. distrito)...")
    
    # Crear 'sexo_clean'. Priorizar '_value_1', usar 'Valor' como fallback.
    map_numeric = {
            1: 'M',
            2: 'F'
            # (Añade otros números si existen)
        }
    
    map_text = {
        'masculino': 'M',
        'femenino': 'F'
        # (Añade otros strings si existen)
    }

    # 1. Mapear la columna numérica (ej. 1 -> 'M')
    sexo_num = df_processed['Diputado.Sexo.Valor'].map(map_numeric)
    
    # 2. Mapear la columna de texto (ej. 'Masculino' -> 'M')
    #    (Usar .str.lower() para asegurar el match)
    sexo_txt = df_processed['Diputado.Sexo._value_1'].str.lower().map(map_text)
    
    # 3. Consolidar: Priorizar la numérica, rellenar con la de texto
    df_processed['sexo_clean'] = sexo_num.fillna(sexo_txt)
    
    # 4. Rellenar cualquier NaN restante
    df_processed['sexo_clean'] = df_processed['sexo_clean'].fillna('Desconocido')
    
    # Crear 'distrito_clean'. Priorizar API, usar BCN como fallback.
    df_processed['distrito_clean'] = df_processed['Distrito.Numero'].fillna(
        df_processed['distrito']
    ).astype('Int64') # Convertir a Entero (que soporta NaNs)

    # Renombrar 'FechaInicio.1' para claridad ANTES de seleccionar
    # (Esto evita errores si hay otras columnas '.1')
    df_processed = df_processed.rename(columns={
        'FechaInicio.1': 'militancia_fecha_inicio',
        'FechaTermino.1': 'militancia_fecha_termino'
    })

    # --- 2. Seleccionar Columnas Finales ---
    logging.info("Seleccionando columnas finales para el archivo procesado...")
    
    COLUMNAS_A_MANTENER = [
        # --- Llaves (Keys) ---
        'Diputado.Id', 
        'Partido.Id', 
        'fuente_periodo', 
        
        # --- Fechas Clave ---
        'FechaInicio',          # Inicio del Período
        'militancia_fecha_inicio', # Inicio Militancia (de 'FechaInicio.1')
        'militancia_fecha_termino',# Fin Militancia (de 'FechaTermino.1')
        
        # --- Identificadores ---
        'nombre_completo',
        'Diputado.Nombre',
        'Diputado.ApellidoPaterno',
        'Diputado.ApellidoMaterno',
        'sexo_clean',           # <-- Columna consolidada
        'Diputado.FechaDefucion',
        
        # --- Features Biográficas (Limpias) ---
        'fecha_nac_clean',
        'edad',
        'rango_etario',
        'estado_civil_clean',
        'numero_total_hijos',
        'pais_nac',
        'ciudad_nac',
        
        # --- Features Legislativas (Limpias) ---
        'distrito_clean',       # <-- Columna consolidada
        
        # --- Features de Educación (Limpias) ---
        'educacion_nivel_clean',
        'universidad_clean',
        'universidad_tipo',
        'carrera_clean_1',
        'carrera_clean_2',
        
        # --- Llaves de Merge (Externas) ---
        'colegio_egreso_merge_key',

        # --- Features Opcionales (Auditoría/Socio-económico) ---
        'padre',
        'madre',
        'colegio_egreso_raw' 
    ]
    
    # (Filtrar por las que existen en el DataFrame)
    columnas_validas = [col for col in COLUMNAS_A_MANTENER if col in df_processed.columns]
    df_final = df_processed[columnas_validas]

    # --- 3. Renombrar Columnas (Normalizar) ---
    logging.info("Normalizando nombres de columnas a 'snake_case'...")
    
    FINAL_RENAME_MAP = {
        # Llaves
        'Diputado.Id': 'diputado_id',
        'Partido.Id': 'partido_id',
        'fuente_periodo': 'periodo',
        
        # Fechas Clave
        'FechaInicio': 'periodo_fecha_inicio',
        'FechaTermino': 'periodo_fecha_termino',
        'militancia_fecha_inicio': 'militancia_fecha_inicio',
        'militancia_fecha_termino': 'militancia_fecha_termino',
        
        # Identificadores
        'nombre_completo': 'nombre_completo',
        'url_bcn': 'url_bcn',
        'Diputado.Nombre': 'nombre',
        'Diputado.ApellidoPaterno': 'apellido_paterno',
        'Diputado.ApellidoMaterno': 'apellido_materno',
        'sexo_clean': 'sexo',
        'Diputado.FechaDefucion': 'fecha_defuncion',
        
        # Features Biográficas
        'fecha_nac_clean': 'fecha_nacimiento',
        'edad': 'edad_inicio_periodo',
        'rango_etario': 'rango_etario',
        'estado_civil_clean': 'estado_civil',
        'numero_total_hijos': 'numero_hijos',
        'pais_nac': 'pais_nacimiento',
        'ciudad_nac': 'ciudad_nacimiento',
        'padre': 'nombre_padre_raw', # (Marcado como 'raw')
        'madre': 'nombre_madre_raw',
        
        # Features Legislativas
        'distrito_clean': 'distrito',
        
        # Features de Educación
        'educacion_nivel_clean': 'educacion_nivel',
        'universidad_clean': 'universidad',
        'universidad_tipo': 'universidad_tipo',
        'carrera_clean_1': 'carrera_primaria',
        'carrera_clean_2': 'carrera_secundaria',
        
        # Llaves de Merge
        'colegio_egreso_merge_key': 'colegio_merge_key',
        'colegio_egreso_raw': 'colegio_raw',
        
        # Texto Crudo
        'bio_texto_completo': 'biografia_texto'
    }

    df_final_renombrado = df_final.rename(columns=FINAL_RENAME_MAP)

    # --- 4. Guardar ---
    try:
        df_final_renombrado.to_parquet(OUTPUT_FILE, index=False)
        logging.info(f"Guardado exitosamente: {OUTPUT_FILE}")
        logging.info(f"Dimensiones del DataFrame maestro: {df_final_renombrado.shape}")
        
        print("\n--- Columnas Finales del DataFrame Limpio ---")
        print(df_final_renombrado.columns.tolist())
        
    except Exception as e:
        logging.error(f"ERROR al guardar en Parquet: {e}")

    display(df_final_renombrado.head())
else:
    logging.error("DataFrame procesado vacío. No se guardó ningún archivo.")

logging.info("--- Procesamiento de Diputados-Período finalizado ---")

2025-10-29 11:20:34,425 - INFO - Consolidando columnas duplicadas (ej. distrito)...
2025-10-29 11:20:34,433 - INFO - Seleccionando columnas finales para el archivo procesado...
2025-10-29 11:20:34,436 - INFO - Normalizando nombres de columnas a 'snake_case'...
2025-10-29 11:20:34,601 - INFO - Guardado exitosamente: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed\diputados_master_clean.parquet
2025-10-29 11:20:34,602 - INFO - Dimensiones del DataFrame maestro: (929, 29)



--- Columnas Finales del DataFrame Limpio ---
['diputado_id', 'partido_id', 'periodo', 'periodo_fecha_inicio', 'militancia_fecha_inicio', 'militancia_fecha_termino', 'nombre_completo', 'nombre', 'apellido_paterno', 'apellido_materno', 'sexo', 'fecha_defuncion', 'fecha_nacimiento', 'edad_inicio_periodo', 'rango_etario', 'estado_civil', 'numero_hijos', 'pais_nacimiento', 'ciudad_nacimiento', 'distrito', 'educacion_nivel', 'universidad', 'universidad_tipo', 'carrera_primaria', 'carrera_secundaria', 'colegio_merge_key', 'nombre_padre_raw', 'nombre_madre_raw', 'colegio_raw']


,diputado_id,partido_id,periodo,periodo_fecha_inicio,militancia_fecha_inicio,militancia_fecha_termino,nombre_completo,nombre,apellido_paterno,apellido_materno,...,distrito,educacion_nivel,universidad,universidad_tipo,carrera_primaria,carrera_secundaria,colegio_merge_key,nombre_padre_raw,nombre_madre_raw,colegio_raw
0,1,DC,1998-2002,2002-03-10,1990-03-11,1994-03-10 23:59:59,Mario Acuña Cisternas,Mario,Acuña,Cisternas,...,52,Universitaria,Universidad Austral de Chile,Universidad Chilena,Agronomía / Veterinaria,None,NaN,Juan Manuel Acuña Ribilar,María Cisterna Fuentealba,None
1,3,RN,1998-2002,2002-03-10,1998-03-11,2002-03-10 23:59:59,Gustavo Alessandri Valdés,Gustavo,Alessandri,Valdés,...,20,Universitaria,Universidad Extranjera,Universidad Extranjera,Agronomía / Veterinaria,None,colegio de los sagrados corazones de manquehue,Gustavo Alessandri Valdés,Verónica Balmaceda,Colegio de los Sagrados Corazones de Manquehue
2,8,DC,1998-2002,2002-03-10,1998-03-11,2002-03-10 23:59:59,Rafael Arratia Valdebenito,Rafael,Arratia,Valdebenito,...,35,Universitaria,Universidad de Chile,Universidad Chilena,Salud,None,colegio manuel leon prado,Rafael Arratia Vidal,Olivia Valdebenito Cuevas,Colegio Manuel León Prado
3,10,PPD,1998-2002,2002-03-10,1998-03-11,2002-03-10 23:59:59,Nelson Avila Contreras,Nelson,Avila,Contreras,...,11,Universitaria,Universidad de Chile,Universidad Chilena,Administración Pública,None,internado nacional barros arana,Pedro Nolasco Ávila Ávila,María Olivia Contreras Chinchón,Internado Nacional Barros Arana
4,11,UDI,1998-2002,2002-03-10,1998-03-11,2002-03-10 23:59:59,Francisco Bartolucci Johnston,Francisco,Bartolucci,Johnston,...,13,Universitaria,Universidad Católica de Valparaíso,Universidad Chilena,Derecho,None,colegio san pedro nolasco,Mario Bartolucci Matera,Josefina Jhonston Miranda,Colegio San Pedro Nolasco


2025-10-29 11:20:34,618 - INFO - --- Procesamiento de Diputados-Período finalizado ---
